In [1]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from keras.applications.vgg16 import VGG16 
import os;

2024-05-22 15:38:53.895233: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 15:38:53.971720: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 15:38:54.229235: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 15:38:55.572373: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# Function to extract features from a frame
def extract_features(frame, model):
    frame = cv2.resize(frame, (224, 224))
    imgx = np.expand_dims(frame, axis = 0)
    features = model.predict(imgx)
    return features.flatten()